In [1]:
# -*- encoding:urf-8 -*-
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table like and matrices
import pandas as pd
import numpy as np

# Modeling Helper
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno

# Configure visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 8,6

poi_path = '../../UAI_data/input/poi_re.csv'
train_Aug_path = '../../UAI_data/input/train_Aug_re.csv'
train_Jul_path = '../../UAI_data/input/train_July_re.csv'
train_jul_demand_path = '../../UAI_data/input/tain_jul_demand.csv'
train_aug_demand_path = '../../UAI_data/input/tain_aug_demand.csv'
location_path = '../../UAI_data/input/location_cls.csv'
holiday_path = '../../UAI_data/input/holiday.csv'
weather_path = '../../UAI_data/input/weather.csv'
test_path = '../../UAI_data/input/test.csv'

poi = pd.read_csv(poi_path,encoding='gbk')
train_aug = pd.read_csv(train_Aug_path,encoding='gbk')
train_jul = pd.read_csv(train_Jul_path,encoding='gbk')
train_jul_demand = pd.read_csv(train_jul_demand_path,encoding='gbk')
train_aug_demand = pd.read_csv(train_aug_demand_path,encoding='gbk')
loc_cls = pd.read_csv(location_path)
holiday = pd.read_csv(holiday_path)
weather = pd.read_csv(weather_path)
test = pd.read_csv(test_path,encoding='gbk')

###  Helper Functions

In [2]:
def plot_histograms( df , variables , n_rows , n_cols ):
    fig = plt.figure( figsize = ( 16 , 12 ) )
    for i, var_name in enumerate( variables ):
        ax=fig.add_subplot( n_rows , n_cols , i+1 )
        df[ var_name ].hist( bins=10 , ax=ax )
        ax.set_title( 'Skew: ' + str( round( float( df[ var_name ].skew() ) , ) ) ) # + ' ' + var_name ) #var_name+" Distribution")
        ax.set_xticklabels( [] , visible=False )
        ax.set_yticklabels( [] , visible=False )
    fig.tight_layout()  # Improves appearance a bit.
    plt.show()

def plot_distribution( df , var , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , hue=target , aspect=4 , row = row , col = col )
    facet.map( sns.kdeplot , var , shade= True )
    facet.set( xlim=( 0 , df[ var ].max() ) )
    facet.add_legend()

def plot_categories( df , cat , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , row = row , col = col )
    facet.map( sns.barplot , cat , target )
    facet.add_legend()

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

def describe_more( df ):
    var = [] ; l = [] ; t = []
    for x in df:
        var.append( x )
        l.append( len( pd.value_counts( df[ x ] ) ) )
        t.append( df[ x ].dtypes )
    levels = pd.DataFrame( { 'Variable' : var , 'Levels' : l , 'Datatype' : t } )
    levels.sort_values( by = 'Levels' , inplace = True )
    return levels

def plot_variable_importance( X , y ):
    tree = DecisionTreeClassifier( random_state = 99 )
    tree.fit( X , y )
    plot_model_var_imp( tree , X , y )
    
def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))

In [3]:
# train_jul_exc = train_jul[train_jul['driver_id'] != 2].copy()
train_jul_exc = train_jul.copy()
coord_jul = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['start_geo_id'].agg({'demand_count':'count'})
coord_jul.loc[:,'estimate_money_mean'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_money'].mean()['estimate_money']
# coord_jul.loc[:,'estimate_money_median'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_money'].median()['estimate_money']
# coord_jul.loc[:,'estimate_money_max'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_money'].max()['estimate_money']
# coord_jul.loc[:,'estimate_money_min'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_money'].min()['estimate_money']
coord_jul.loc[:,'estimate_distance_mean'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_distance'].mean()['estimate_distance']
# coord_jul.loc[:,'estimate_distance_median'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_distance'].median()['estimate_distance']
# coord_jul.loc[:,'estimate_distance_max'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_distance'].max()['estimate_distance']
# coord_jul.loc[:,'estimate_distance_min'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_distance'].min()['estimate_distance']
coord_jul.loc[:,'estimate_term_mean'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_term'].mean()['estimate_term']
# coord_jul.loc[:,'estimate_term_median'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_term'].median()['estimate_term']
# coord_jul.loc[:,'estimate_term_max'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_term'].max()['estimate_term']
# coord_jul.loc[:,'estimate_term_min'] = train_jul_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_term'].min()['estimate_term']

# coord_jul.to_csv('train_jul_exc.csv',index=False)
coord_jul

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,estimate_money_mean,estimate_distance_mean,estimate_term_mean
0,2017-07-01,0,0,10,1,101.000000,14431.000000,24.000000
1,2017-07-01,0,0,21,1,39.000000,4905.000000,9.000000
2,2017-07-01,0,0,64,1,78.860000,8764.000000,9.000000
3,2017-07-01,0,1,1,2,28.500000,2437.000000,4.500000
4,2017-07-01,0,1,15,2,67.000000,10955.000000,21.500000
5,2017-07-01,0,1,37,1,63.000000,10010.000000,20.000000
6,2017-07-01,0,1,39,2,67.110000,11493.000000,17.500000
7,2017-07-01,0,1,89,2,59.675000,9208.500000,18.000000
8,2017-07-01,0,1,95,1,90.000000,15677.000000,31.000000
9,2017-07-01,0,1,160,2,46.520000,6347.000000,12.000000


In [4]:
test

,test_id,create_date,create_hour,start_geo_id,end_geo_id
0,0,2017/8/1,21,24,27
1,1,2017/8/1,21,24,9
2,2,2017/8/1,21,24,22
3,3,2017/8/1,17,24,9
4,4,2017/8/1,17,55,55
5,5,2017/8/1,17,110,88
6,6,2017/8/1,9,23,21
7,7,2017/8/1,21,16,50
8,8,2017/8/1,13,6,24
9,9,2017/8/1,21,6,22


In [5]:
train_aug_demand

,create_date,create_hour,start_geo_id,end_geo_id,demand_count
0,2017-08-01,0,1,1,1
1,2017-08-01,0,1,9,1
2,2017-08-01,0,1,41,1
3,2017-08-01,0,1,73,1
4,2017-08-01,0,1,101,1
5,2017-08-01,0,1,191,1
6,2017-08-01,0,1,198,1
7,2017-08-01,0,3,56,1
8,2017-08-01,0,5,1,1
9,2017-08-01,0,5,5,3


In [6]:
test['create_hour'].value_counts()

22    443
20    411
21    383
17    346
13    334
9     313
18    261
15    254
8     246
16    242
11    240
7     234
19    225
10    211
14    201
12    180
23    147
6      76
0      70
1      61
5      47
3      30
2      25
4      20
Name: create_hour, dtype: int64

### Valid set split

In [7]:
# train_aug_exc = train_aug[train_aug['driver_id'] != 2].copy()
train_aug_exc = train_aug.copy()
coord_aug = train_aug_exc.groupby(['create_date','create_hour','start_geo_id','end_geo_id'],as_index=False)['start_geo_id'].agg({'demand_count':'count'})
# coord_aug.to_csv('train_aug_exc.csv',index=False)
coord_aug

,create_date,create_hour,start_geo_id,end_geo_id,demand_count
0,2017-08-01,0,1,1,1
1,2017-08-01,0,1,9,1
2,2017-08-01,0,1,41,1
3,2017-08-01,0,1,73,1
4,2017-08-01,0,1,101,1
5,2017-08-01,0,1,191,1
6,2017-08-01,0,1,198,1
7,2017-08-01,0,3,56,1
8,2017-08-01,0,5,1,1
9,2017-08-01,0,5,5,3


In [8]:
dict_list = {
'l_50' : [0,1,2,3,4,5],
'l_100' : [6],
'l_150' : [12,23],
'l_200' : [10,14,19],
'l_250' : [7,8,11,15,16,18],
'l_300' : [9,13],
'l_350' : [17],
'l_400' : [20,21]
# 'l_450' : [22]
}
l_22 = coord_aug[coord_aug['create_hour'] == 22].index
r = np.random.choice(l_22,450)
valid = coord_aug.iloc[r]
for key,l in dict_list.items():
    num =  int(key.split('_')[1])
    for hour in l:
        l_tmp = coord_aug[coord_aug['create_hour'] == hour].index
        r = np.random.choice(l_tmp,num)
        tmp_valid = coord_aug.iloc[r].copy()
        valid = pd.concat([valid,tmp_valid],axis=0)
valid.sort_values(['create_date','create_hour'],inplace=True)
valid

,create_date,create_hour,start_geo_id,end_geo_id,demand_count
100,2017-08-01,0,22,39,1
68,2017-08-01,0,17,128,2
96,2017-08-01,0,22,22,2
2,2017-08-01,0,1,41,1
375,2017-08-01,2,88,51,1
328,2017-08-01,2,24,2,1
299,2017-08-01,2,1,26,1
338,2017-08-01,2,24,88,1
350,2017-08-01,2,33,39,1
301,2017-08-01,2,1,99,1


In [9]:
valid['create_hour'].unique()

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22,  1,  3,  5,  7,  9,
       11, 13, 15, 17, 19, 21, 23], dtype=int64)

## Feature Extracting
### 1. order list attribution

In [10]:
train_tr = coord_jul.copy()
valid_tr = valid.copy()
train_tr
# valid_tr
# valid_tr['demand_count']

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,estimate_money_mean,estimate_distance_mean,estimate_term_mean
0,2017-07-01,0,0,10,1,101.000000,14431.000000,24.000000
1,2017-07-01,0,0,21,1,39.000000,4905.000000,9.000000
2,2017-07-01,0,0,64,1,78.860000,8764.000000,9.000000
3,2017-07-01,0,1,1,2,28.500000,2437.000000,4.500000
4,2017-07-01,0,1,15,2,67.000000,10955.000000,21.500000
5,2017-07-01,0,1,37,1,63.000000,10010.000000,20.000000
6,2017-07-01,0,1,39,2,67.110000,11493.000000,17.500000
7,2017-07-01,0,1,89,2,59.675000,9208.500000,18.000000
8,2017-07-01,0,1,95,1,90.000000,15677.000000,31.000000
9,2017-07-01,0,1,160,2,46.520000,6347.000000,12.000000


In [11]:
# time attribution
holiday['create_date'] = pd.to_datetime(holiday['create_date'])
train_tr['create_date'] = pd.to_datetime(train_tr['create_date'])
valid_tr['create_date'] = pd.to_datetime(valid_tr['create_date'])
train_tr = pd.merge(train_tr,holiday,on='create_date',how='left')
valid_tr = pd.merge(valid_tr,holiday,on='create_date',how='left')
train_tr['dayOfWeek'] = train_tr['create_date'].dt.dayofweek
valid_tr['dayOfWeek'] = valid_tr['create_date'].dt.dayofweek
train_tr

# # time attribution
# holiday['create_date'] = pd.to_datetime(holiday['create_date'])
# train_tr['create_date'] = pd.to_datetime(train_tr['create_date'])
# test['create_date'] = pd.to_datetime(test['create_date'])
# train_tr = pd.merge(train_tr,holiday,on='create_date',how='left')
# test = pd.merge(test,holiday,on='create_date',how='left')
# train_tr['dayOfWeek'] = train_tr['create_date'].dt.dayofweek
# test['dayOfWeek'] = test['create_date'].dt.dayofweek
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,estimate_money_mean,estimate_distance_mean,estimate_term_mean,holiday,dayOfWeek
0,2017-07-01,0,0,10,1,101.000000,14431.000000,24.000000,1,5
1,2017-07-01,0,0,21,1,39.000000,4905.000000,9.000000,1,5
2,2017-07-01,0,0,64,1,78.860000,8764.000000,9.000000,1,5
3,2017-07-01,0,1,1,2,28.500000,2437.000000,4.500000,1,5
4,2017-07-01,0,1,15,2,67.000000,10955.000000,21.500000,1,5
5,2017-07-01,0,1,37,1,63.000000,10010.000000,20.000000,1,5
6,2017-07-01,0,1,39,2,67.110000,11493.000000,17.500000,1,5
7,2017-07-01,0,1,89,2,59.675000,9208.500000,18.000000,1,5
8,2017-07-01,0,1,95,1,90.000000,15677.000000,31.000000,1,5
9,2017-07-01,0,1,160,2,46.520000,6347.000000,12.000000,1,5


In [12]:
coord_money_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_money_mean'].agg({'money_hour_mean':'mean'})
coord_money_day = train_tr.groupby(['dayOfWeek','start_geo_id','end_geo_id'],as_index=False)['estimate_money_mean'].agg({'money_day_mean':'mean'})
coord_dis_hour = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['estimate_distance_mean'].agg({'dis_hour_mean':'mean'})
coord_term_hour = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['estimate_term_mean'].agg({'term_hour_mean':'mean'})
train_tr = pd.merge(train_tr,coord_money_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
train_tr = pd.merge(train_tr,coord_money_day,on=['dayOfWeek','start_geo_id','end_geo_id'],how='left')
train_tr = pd.merge(train_tr,coord_dis_hour,on=['start_geo_id','end_geo_id'],how='left')
train_tr = pd.merge(train_tr,coord_term_hour,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_money_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_money_day,on=['dayOfWeek','start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_dis_hour,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_term_hour,on=['start_geo_id','end_geo_id'],how='left')
valid_tr

# coord_money_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['estimate_money_mean'].agg({'money_hour_mean':'mean'})
# coord_money_day = train_tr.groupby(['dayOfWeek','start_geo_id','end_geo_id'],as_index=False)['estimate_money_mean'].agg({'money_day_mean':'mean'})
# coord_dis_hour = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['estimate_distance_mean'].agg({'dis_hour_mean':'mean'})
# coord_term_hour = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['estimate_term_mean'].agg({'term_hour_mean':'mean'})
# train_tr = pd.merge(train_tr,coord_money_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# # train_tr = pd.merge(train_tr,coord_money_day,on=['dayOfWeek','start_geo_id','end_geo_id'],how='left')
# train_tr = pd.merge(train_tr,coord_dis_hour,on=['start_geo_id','end_geo_id'],how='left')
# train_tr = pd.merge(train_tr,coord_term_hour,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_money_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_money_day,on=['dayOfWeek','start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_dis_hour,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_term_hour,on=['start_geo_id','end_geo_id'],how='left')
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,term_hour_mean
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,8.000088
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,29.750000
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,4.480016
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,40.047128
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,25.766667
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,25.451156
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,21.973866
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,27.764229
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,13.987179
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,26.503876


### 2.Location
#### 2.1 Pure location

In [13]:
loc_start = loc_cls.copy()
loc_start.rename(columns={'location_id':'start_geo_id','cluster':'start_cluster'},inplace=True)
train_tr = pd.merge(train_tr,loc_start,on='start_geo_id',how='left')
loc_end = loc_cls.copy()
loc_end.rename(columns={'location_id':'end_geo_id','cluster':'end_cluster'},inplace=True)
train_tr = pd.merge(train_tr,loc_end,on='end_geo_id',how='left')
loc_start = loc_cls.copy()
loc_start.rename(columns={'location_id':'start_geo_id','cluster':'start_cluster'},inplace=True)
valid_tr = pd.merge(valid_tr,loc_start,on='start_geo_id',how='left')
loc_end = loc_cls.copy()
loc_end.rename(columns={'location_id':'end_geo_id','cluster':'end_cluster'},inplace=True)
valid_tr = pd.merge(valid_tr,loc_end,on='end_geo_id',how='left')
# valid_tr['end_cluster'].astype('int',inplace=True)
valid_tr

# loc_start = loc_cls.copy()
# loc_start.rename(columns={'location_id':'start_geo_id','cluster':'start_cluster'},inplace=True)
# train_tr = pd.merge(train_tr,loc_start,on='start_geo_id',how='left')
# loc_end = loc_cls.copy()
# loc_end.rename(columns={'location_id':'end_geo_id','cluster':'end_cluster'},inplace=True)
# train_tr = pd.merge(train_tr,loc_end,on='end_geo_id',how='left')
# loc_start = loc_cls.copy()
# loc_start.rename(columns={'location_id':'start_geo_id','cluster':'start_cluster'},inplace=True)
# test = pd.merge(test,loc_start,on='start_geo_id',how='left')
# loc_end = loc_cls.copy()
# loc_end.rename(columns={'location_id':'end_geo_id','cluster':'end_cluster'},inplace=True)
# test = pd.merge(test,loc_end,on='end_geo_id',how='left')
# # test['end_cluster'].astype('int',inplace=True)
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,term_hour_mean,start_cluster,end_cluster
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,8.000088,5.0,1.0
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,29.750000,5.0,3.0
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,4.480016,5.0,5.0
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,40.047128,1.0,5.0
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,25.766667,5.0,1.0
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,25.451156,1.0,1.0
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,21.973866,1.0,5.0
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,27.764229,1.0,5.0
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,13.987179,5.0,1.0
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,26.503876,1.0,5.0


In [14]:
coord_se_mean = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_mean':'mean'})
coord_se_median = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_median':'median'})
coord_se_max = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_max':'max'})
coord_se_min = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_min':'min'})
train_tr = pd.merge(train_tr,coord_se_mean,on=['start_geo_id','end_geo_id'],how='left')
train_tr = pd.merge(train_tr,coord_se_median,on=['start_geo_id','end_geo_id'],how='left')
train_tr = pd.merge(train_tr,coord_se_max,on=['start_geo_id','end_geo_id'],how='left')
train_tr = pd.merge(train_tr,coord_se_min,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_mean,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_median,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_max,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_min,on=['start_geo_id','end_geo_id'],how='left')
valid_tr

# coord_se_mean = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_mean':'mean'})
# coord_se_median = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_median':'median'})
# coord_se_max = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_max':'max'})
# coord_se_min = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_min':'min'})
# train_tr = pd.merge(train_tr,coord_se_mean,on=['start_geo_id','end_geo_id'],how='left')
# train_tr = pd.merge(train_tr,coord_se_median,on=['start_geo_id','end_geo_id'],how='left')
# train_tr = pd.merge(train_tr,coord_se_max,on=['start_geo_id','end_geo_id'],how='left')
# train_tr = pd.merge(train_tr,coord_se_min,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_se_mean,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_se_median,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_se_max,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_se_min,on=['start_geo_id','end_geo_id'],how='left')
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,term_hour_mean,start_cluster,end_cluster,sloc_eloc_mean,sloc_eloc_median,sloc_eloc_max,sloc_eloc_min
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,8.000088,5.0,1.0,2.940397,2.0,25.0,1.0
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,29.750000,5.0,3.0,1.500000,1.0,3.0,1.0
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,4.480016,5.0,5.0,4.448598,3.0,56.0,1.0
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,40.047128,1.0,5.0,1.619048,1.0,9.0,1.0
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,25.766667,5.0,1.0,1.490909,1.0,6.0,1.0
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,25.451156,1.0,1.0,2.057143,1.0,21.0,1.0
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,21.973866,1.0,5.0,1.865079,1.0,9.0,1.0
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,27.764229,1.0,5.0,1.732484,1.0,16.0,1.0
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,13.987179,5.0,1.0,1.346154,1.0,6.0,1.0
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,26.503876,1.0,5.0,1.325581,1.0,6.0,1.0


In [15]:
coord_se_mean = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_mean':'mean'})
coord_se_median = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_median':'median'})
coord_se_max = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_max':'max'})
coord_se_min = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_min':'min'})
train_tr = pd.merge(train_tr,coord_se_mean,on=['start_cluster','end_cluster'],how='left')
train_tr = pd.merge(train_tr,coord_se_median,on=['start_cluster','end_cluster'],how='left')
train_tr = pd.merge(train_tr,coord_se_max,on=['start_cluster','end_cluster'],how='left')
train_tr = pd.merge(train_tr,coord_se_min,on=['start_cluster','end_cluster'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_mean,on=['start_cluster','end_cluster'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_median,on=['start_cluster','end_cluster'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_max,on=['start_cluster','end_cluster'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_min,on=['start_cluster','end_cluster'],how='left')
valid_tr

# coord_se_mean = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_mean':'mean'})
# coord_se_median = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_median':'median'})
# coord_se_max = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_max':'max'})
# coord_se_min = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloccl_eloccl_min':'min'})
# train_tr = pd.merge(train_tr,coord_se_mean,on=['start_cluster','end_cluster'],how='left')
# train_tr = pd.merge(train_tr,coord_se_median,on=['start_cluster','end_cluster'],how='left')
# train_tr = pd.merge(train_tr,coord_se_max,on=['start_cluster','end_cluster'],how='left')
# train_tr = pd.merge(train_tr,coord_se_min,on=['start_cluster','end_cluster'],how='left')
# test = pd.merge(test,coord_se_mean,on=['start_cluster','end_cluster'],how='left')
# test = pd.merge(test,coord_se_median,on=['start_cluster','end_cluster'],how='left')
# test = pd.merge(test,coord_se_max,on=['start_cluster','end_cluster'],how='left')
# test = pd.merge(test,coord_se_min,on=['start_cluster','end_cluster'],how='left')
# test



,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,...,start_cluster,end_cluster,sloc_eloc_mean,sloc_eloc_median,sloc_eloc_max,sloc_eloc_min,sloccl_eloccl_mean,sloccl_eloccl_median,sloccl_eloccl_max,sloccl_eloccl_min
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,...,5.0,1.0,2.940397,2.0,25.0,1.0,1.928592,1.0,144.0,1.0
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,...,5.0,3.0,1.500000,1.0,3.0,1.0,1.414443,1.0,20.0,1.0
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,...,5.0,5.0,4.448598,3.0,56.0,1.0,1.854976,1.0,73.0,1.0
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,...,1.0,5.0,1.619048,1.0,9.0,1.0,2.068860,1.0,198.0,1.0
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,...,5.0,1.0,1.490909,1.0,6.0,1.0,1.928592,1.0,144.0,1.0
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,...,1.0,1.0,2.057143,1.0,21.0,1.0,2.311007,1.0,178.0,1.0
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,...,1.0,5.0,1.865079,1.0,9.0,1.0,2.068860,1.0,198.0,1.0
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,...,1.0,5.0,1.732484,1.0,16.0,1.0,2.068860,1.0,198.0,1.0
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,...,5.0,1.0,1.346154,1.0,6.0,1.0,1.928592,1.0,144.0,1.0
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,...,1.0,5.0,1.325581,1.0,6.0,1.0,2.068860,1.0,198.0,1.0


In [16]:
coord_se_sum = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_sum':'sum'})
coord_s_sum = coord_se_sum.groupby('start_geo_id',as_index=False)['sloc_eloc_sum'].agg({'sloc_sum':'sum'})
coord_e_sum = coord_se_sum.groupby('end_geo_id',as_index=False)['sloc_eloc_sum'].agg({'eloc_sum':'sum'})
coord_se_sum = pd.merge(coord_se_sum,coord_s_sum,on='start_geo_id',how='left')
coord_se_sum = pd.merge(coord_se_sum,coord_e_sum,on='end_geo_id',how='left')
coord_se_sum.loc[:,'se_start_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['sloc_sum'])
coord_se_sum.loc[:,'se_end_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['eloc_sum'])
del coord_se_sum['sloc_eloc_sum'],coord_se_sum['sloc_sum'],coord_se_sum['eloc_sum']
# coord_se_sum
train_tr = pd.merge(train_tr,coord_se_sum,on=['start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_sum,on=['start_geo_id','end_geo_id'],how='left')
valid_tr

# coord_se_sum = train_tr.groupby(['start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'sloc_eloc_sum':'sum'})
# coord_s_sum = coord_se_sum.groupby('start_geo_id',as_index=False)['sloc_eloc_sum'].agg({'sloc_sum':'sum'})
# coord_e_sum = coord_se_sum.groupby('end_geo_id',as_index=False)['sloc_eloc_sum'].agg({'eloc_sum':'sum'})
# coord_se_sum = pd.merge(coord_se_sum,coord_s_sum,on='start_geo_id',how='left')
# coord_se_sum = pd.merge(coord_se_sum,coord_e_sum,on='end_geo_id',how='left')
# coord_se_sum.loc[:,'se_start_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['sloc_sum'])
# coord_se_sum.loc[:,'se_end_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['eloc_sum'])
# del coord_se_sum['sloc_eloc_sum'],coord_se_sum['sloc_sum'],coord_se_sum['eloc_sum']
# # coord_se_sum
# train_tr = pd.merge(train_tr,coord_se_sum,on=['start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_se_sum,on=['start_geo_id','end_geo_id'],how='left')
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,...,sloc_eloc_mean,sloc_eloc_median,sloc_eloc_max,sloc_eloc_min,sloccl_eloccl_mean,sloccl_eloccl_median,sloccl_eloccl_max,sloccl_eloccl_min,se_start_ratio,se_end_ratio
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,...,2.940397,2.0,25.0,1.0,1.928592,1.0,144.0,1.0,0.046675,0.058108
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,...,1.500000,1.0,3.0,1.0,1.414443,1.0,20.0,1.0,0.000460,0.012605
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,...,4.448598,3.0,56.0,1.0,1.854976,1.0,73.0,1.0,0.083398,0.089269
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,...,1.619048,1.0,9.0,1.0,2.068860,1.0,198.0,1.0,0.008479,0.008872
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,...,1.490909,1.0,6.0,1.0,1.928592,1.0,144.0,1.0,0.008226,0.010196
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,...,2.057143,1.0,21.0,1.0,2.311007,1.0,178.0,1.0,0.002497,0.062096
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,...,1.865079,1.0,9.0,1.0,2.068860,1.0,198.0,1.0,0.011721,0.016668
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,...,1.732484,1.0,16.0,1.0,2.068860,1.0,198.0,1.0,0.004717,0.025612
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,...,1.346154,1.0,6.0,1.0,1.928592,1.0,144.0,1.0,0.030043,0.007634
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,...,1.325581,1.0,6.0,1.0,2.068860,1.0,198.0,1.0,0.002843,0.026611


In [17]:
coord_se_sum = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloc_eloc_sum':'sum'})
coord_s_sum = coord_se_sum.groupby('start_cluster',as_index=False)['sloc_eloc_sum'].agg({'sloc_sum':'sum'})
coord_e_sum = coord_se_sum.groupby('end_cluster',as_index=False)['sloc_eloc_sum'].agg({'eloc_sum':'sum'})
coord_se_sum = pd.merge(coord_se_sum,coord_s_sum,on='start_cluster',how='left')
coord_se_sum = pd.merge(coord_se_sum,coord_e_sum,on='end_cluster',how='left')
coord_se_sum.loc[:,'se_start_cls_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['sloc_sum'])
coord_se_sum.loc[:,'se_end_cls_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['eloc_sum'])
del coord_se_sum['sloc_eloc_sum'],coord_se_sum['sloc_sum'],coord_se_sum['eloc_sum']
# coord_se_sum
train_tr = pd.merge(train_tr,coord_se_sum,on=['start_cluster','end_cluster'],how='left')
valid_tr = pd.merge(valid_tr,coord_se_sum,on=['start_cluster','end_cluster'],how='left')
valid_tr

# coord_se_sum = train_tr.groupby(['start_cluster','end_cluster'],as_index=False)['demand_count'].agg({'sloc_eloc_sum':'sum'})
# coord_s_sum = coord_se_sum.groupby('start_cluster',as_index=False)['sloc_eloc_sum'].agg({'sloc_sum':'sum'})
# coord_e_sum = coord_se_sum.groupby('end_cluster',as_index=False)['sloc_eloc_sum'].agg({'eloc_sum':'sum'})
# coord_se_sum = pd.merge(coord_se_sum,coord_s_sum,on='start_cluster',how='left')
# coord_se_sum = pd.merge(coord_se_sum,coord_e_sum,on='end_cluster',how='left')
# coord_se_sum.loc[:,'se_start_cls_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['sloc_sum'])
# coord_se_sum.loc[:,'se_end_cls_ratio'] = coord_se_sum['sloc_eloc_sum'] / (1.0 * coord_se_sum['eloc_sum'])
# del coord_se_sum['sloc_eloc_sum'],coord_se_sum['sloc_sum'],coord_se_sum['eloc_sum']
# # coord_se_sum
# train_tr = pd.merge(train_tr,coord_se_sum,on=['start_cluster','end_cluster'],how='left')
# test = pd.merge(test,coord_se_sum,on=['start_cluster','end_cluster'],how='left')
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,...,sloc_eloc_max,sloc_eloc_min,sloccl_eloccl_mean,sloccl_eloccl_median,sloccl_eloccl_max,sloccl_eloccl_min,se_start_ratio,se_end_ratio,se_start_cls_ratio,se_end_cls_ratio
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,...,25.0,1.0,1.928592,1.0,144.0,1.0,0.046675,0.058108,0.263846,0.518449
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,...,3.0,1.0,1.414443,1.0,20.0,1.0,0.000460,0.012605,0.029368,0.504273
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,...,56.0,1.0,1.854976,1.0,73.0,1.0,0.083398,0.089269,0.579441,0.566873
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,...,9.0,1.0,2.068860,1.0,198.0,1.0,0.008479,0.008872,0.539159,0.289222
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,...,6.0,1.0,1.928592,1.0,144.0,1.0,0.008226,0.010196,0.263846,0.518449
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,...,21.0,1.0,2.311007,1.0,178.0,1.0,0.002497,0.062096,0.317524,0.342112
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,...,9.0,1.0,2.068860,1.0,198.0,1.0,0.011721,0.016668,0.539159,0.289222
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,...,16.0,1.0,2.068860,1.0,198.0,1.0,0.004717,0.025612,0.539159,0.289222
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,...,6.0,1.0,1.928592,1.0,144.0,1.0,0.030043,0.007634,0.263846,0.518449
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,...,6.0,1.0,2.068860,1.0,198.0,1.0,0.002843,0.026611,0.539159,0.289222


### Time 

In [18]:
map_hour = {1:1,2:1,3:1,4:1,5:1,6:1,7:2,8:2,9:2,10:3,11:3,12:3,13:4,14:4,15:4,16:5,17:5,18:5,19:6,20:6,21:6,22:0,23:0,0:0}
train_tr.loc[:,'hour_cls'] = train_tr['create_hour'].map(lambda x: map_hour[x])
valid_tr.loc[:,'hour_cls'] = valid_tr['create_hour'].map(lambda x: map_hour[x])
valid_tr

# map_hour = {1:1,2:1,3:1,4:1,5:1,6:1,7:2,8:2,9:2,10:3,11:3,12:3,13:4,14:4,15:4,16:5,17:5,18:5,19:6,20:6,21:6,22:0,23:0,0:0}
# train_tr.loc[:,'hour_cls'] = train_tr['create_hour'].map(lambda x: map_hour[x])
# test.loc[:,'hour_cls'] = test['create_hour'].map(lambda x: map_hour[x])
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,...,sloc_eloc_min,sloccl_eloccl_mean,sloccl_eloccl_median,sloccl_eloccl_max,sloccl_eloccl_min,se_start_ratio,se_end_ratio,se_start_cls_ratio,se_end_cls_ratio,hour_cls
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,...,1.0,1.928592,1.0,144.0,1.0,0.046675,0.058108,0.263846,0.518449,0
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,...,1.0,1.414443,1.0,20.0,1.0,0.000460,0.012605,0.029368,0.504273,0
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,...,1.0,1.854976,1.0,73.0,1.0,0.083398,0.089269,0.579441,0.566873,0
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,...,1.0,2.068860,1.0,198.0,1.0,0.008479,0.008872,0.539159,0.289222,0
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,...,1.0,1.928592,1.0,144.0,1.0,0.008226,0.010196,0.263846,0.518449,1
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,...,1.0,2.311007,1.0,178.0,1.0,0.002497,0.062096,0.317524,0.342112,1
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,...,1.0,2.068860,1.0,198.0,1.0,0.011721,0.016668,0.539159,0.289222,1
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,...,1.0,2.068860,1.0,198.0,1.0,0.004717,0.025612,0.539159,0.289222,1
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,...,1.0,1.928592,1.0,144.0,1.0,0.030043,0.007634,0.263846,0.518449,1
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,...,1.0,2.068860,1.0,198.0,1.0,0.002843,0.026611,0.539159,0.289222,1


In [19]:
coord_hour = train_tr.groupby('create_hour', as_index=False)['demand_count'].agg({'demand_count_h_avg':'std'})
coord_hour

,create_hour,demand_count_h_avg
0,0,1.935778
1,1,1.572084
2,2,0.739049
3,3,0.925068
4,4,0.971026
5,5,1.045984
6,6,1.257506
7,7,2.156388
8,8,3.425920
9,9,2.654506


### 3. Location + Time

In [20]:
coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_avg':'mean'})
train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_median':'median'})
train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_max':'max'})
train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_std':'std'})
train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
valid_tr = pd.merge(valid_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
valid_tr

# coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_avg':'mean'})
# train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_median':'median'})
# train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_max':'max'})
# train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# coord_hour = train_tr.groupby(['create_hour','start_geo_id','end_geo_id'],as_index=False)['demand_count'].agg({'demand_count_h_std':'std'})
# train_tr = pd.merge(train_tr,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# test = pd.merge(test,coord_hour,on=['create_hour','start_geo_id','end_geo_id'],how='left')
# test

,create_date,create_hour,start_geo_id,end_geo_id,demand_count,holiday,dayOfWeek,money_hour_mean,money_day_mean,dis_hour_mean,...,sloccl_eloccl_min,se_start_ratio,se_end_ratio,se_start_cls_ratio,se_end_cls_ratio,hour_cls,demand_count_h_avg,demand_count_h_median,demand_count_h_max,demand_count_h_std
0,2017-08-01,0,22,39,1,0,1,40.414660,39.166027,4241.878884,...,1.0,0.046675,0.058108,0.263846,0.518449,0,2.500000,1.0,12.0,3.034120
1,2017-08-01,0,17,128,2,0,1,NaN,88.000000,19667.000000,...,1.0,0.000460,0.012605,0.029368,0.504273,0,NaN,NaN,NaN,NaN
2,2017-08-01,0,22,22,2,0,1,30.946554,31.001616,2540.075593,...,1.0,0.083398,0.089269,0.579441,0.566873,0,3.250000,2.0,28.0,5.998903
3,2017-08-01,0,1,41,1,0,1,126.010000,133.683846,20582.471066,...,1.0,0.008479,0.008872,0.539159,0.289222,0,1.000000,1.0,1.0,NaN
4,2017-08-01,2,88,51,1,0,1,NaN,73.363333,15513.443939,...,1.0,0.008226,0.010196,0.263846,0.518449,1,NaN,NaN,NaN,NaN
5,2017-08-01,2,24,2,1,0,1,NaN,87.551464,13616.525612,...,1.0,0.002497,0.062096,0.317524,0.342112,1,NaN,NaN,NaN,NaN
6,2017-08-01,2,1,26,1,0,1,NaN,75.132375,11392.201905,...,1.0,0.011721,0.016668,0.539159,0.289222,1,NaN,NaN,NaN,NaN
7,2017-08-01,2,24,88,1,0,1,109.000000,104.919444,16327.369851,...,1.0,0.004717,0.025612,0.539159,0.289222,1,1.000000,1.0,1.0,0.000000
8,2017-08-01,2,33,39,1,0,1,NaN,49.568933,7469.746410,...,1.0,0.030043,0.007634,0.263846,0.518449,1,NaN,NaN,NaN,NaN
9,2017-08-01,2,1,99,1,0,1,NaN,86.741000,14417.703488,...,1.0,0.002843,0.026611,0.539159,0.289222,1,NaN,NaN,NaN,NaN


In [21]:
coord = train_tr.groupby(['create_hour','start_geo_id'],as_index=False)['demand_count'].agg({'demand_count_start_avg':'mean'})
coord

,create_hour,start_geo_id,demand_count_start_avg
0,0,0,1.166667
1,0,1,1.304054
2,0,2,1.451613
3,0,3,1.114286
4,0,4,1.000000
5,0,5,1.403175
6,0,6,1.858942
7,0,7,1.000000
8,0,8,1.192771
9,0,9,1.380952


In [22]:
np.setdiff1d(train_tr.columns,valid_tr.columns)
# np.setdiff1d(test.columns,train_tr.columns)

array(['estimate_distance_mean', 'estimate_money_mean',
       'estimate_term_mean'], dtype=object)

### Training

In [23]:
do_not_use_list = ['create_date','demand_count','estimate_distance_mean','estimate_money_mean','estimate_term_mean','test_id']
predictors = [f for f in train_tr.columns if f not in do_not_use_list]
print predictors

['create_hour', 'start_geo_id', 'end_geo_id', 'holiday', 'dayOfWeek', 'money_hour_mean', 'money_day_mean', 'dis_hour_mean', 'term_hour_mean', 'start_cluster', 'end_cluster', 'sloc_eloc_mean', 'sloc_eloc_median', 'sloc_eloc_max', 'sloc_eloc_min', 'sloccl_eloccl_mean', 'sloccl_eloccl_median', 'sloccl_eloccl_max', 'sloccl_eloccl_min', 'se_start_ratio', 'se_end_ratio', 'se_start_cls_ratio', 'se_end_cls_ratio', 'hour_cls', 'demand_count_h_avg', 'demand_count_h_median', 'demand_count_h_max', 'demand_count_h_std']


In [ ]:
import xgboost as xgb
params = {'min_child_weight': 100, 'eta': 0.1, 'colsample_bytree': 0.3, 'max_depth': 7,
                'subsample': 0.8, 'lambda': 1, 'nthread': 4, 'booster' : 'gbtree', 'silent': 1,
                'eval_metric': 'rmse', 'objective': 'reg:linear'}
boostRound = 150

train_tr_trip = train_tr[train_tr['create_date'] >= '2017-07-01'].copy()
# print train_feat_1[predictors]

xgbtrain = xgb.DMatrix(train_tr_trip[predictors], train_tr_trip['demand_count'],missing=np.nan)
# xgbvalid = xgb.DMatrix(test_feat_1[predictors])
model = xgb.train(params, xgbtrain, num_boost_round=boostRound)
param_score = pd.Series(model.get_fscore()).sort_values(ascending=False)

In [ ]:
param_score

In [ ]:
def score(pred,valid):
    pred_l = list(pred)
    valid_l = list(valid)
    sum_ = 0
    for i in range(len(pred_l)):
        sum_ += abs(pred_l[i] - valid_l[i])
    return sum_ / (1.0 * len(pred_l))
#     return np.sum(np.abs(pred - valid)) / (1.0 * len(pred))

In [ ]:
xgbvalid = xgb.DMatrix(valid_tr[predictors],missing=np.nan)
valid_tr.loc[:,'result'] = model.predict(xgbvalid)
valid_tr['result'].fillna(1,inplace=True)
valid_tr['result']

# xgbvalid = xgb.DMatrix(test[predictors],missing=np.nan)
# test.loc[:,'count'] = model.predict(xgbvalid)
# test['count'].fillna(1,inplace=True)
# test['count']

In [ ]:
score(valid_tr['result'],valid_tr['demand_count'])

In [ ]:
# test['test_id'] = test['test_id'].astype('int')
# test[['test_id','count']].to_csv('result.csv',index=False)